# HitPredict 1st Milestone
## Data Collection, Visualisation and Feature Engineering

Hit predict will predict the popularity of a song based on some of its musical properties. We used the Spotify DB dataset from kaggle, which contains numerous rows of features of over 230.000 tracks. It was assembled using Spotify's API.

As always we started with importing the libraries that we'll be using.

In [1]:
import numpy as np 
import pandas as pd 
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
np.random.seed(10)

The database is in .csv format, we used Pandas' read_csv() function to import it to Python. We visualize the data below.

In [2]:
data = pd.read_csv('SpotifyFeatures.csv')
data.head()

,genre,artist_name,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,Movie,Henri Salvador,C'est beau de faire un Show,0BRjO6ga9RKCKjfDqeFgWV,0,0.611,0.389,99373,0.910,0.000,C#,0.3460,-1.828,Major,0.0525,166.969,4/4,0.814
1,Movie,Martin & les fées,Perdu d'avance (par Gad Elmaleh),0BjC1NfoEOOusryehmNudP,1,0.246,0.590,137373,0.737,0.000,F#,0.1510,-5.559,Minor,0.0868,174.003,4/4,0.816
2,Movie,Joseph Williams,Don't Let Me Be Lonely Tonight,0CoSDzoNIKCRs124s9uTVy,3,0.952,0.663,170267,0.131,0.000,C,0.1030,-13.879,Minor,0.0362,99.488,5/4,0.368
3,Movie,Henri Salvador,Dis-moi Monsieur Gordon Cooper,0Gc6TVm52BwZD07Ki6tIvf,0,0.703,0.240,152427,0.326,0.000,C#,0.0985,-12.178,Major,0.0395,171.758,4/4,0.227
4,Movie,Fabien Nataf,Ouverture,0IuslXpMROHdEPvSl1fTQK,4,0.950,0.331,82625,0.225,0.123,F,0.2020,-21.150,Major,0.0456,140.576,4/4,0.390


Track ID is not giving us any useful information so lets just get rid of it

In [3]:
del data['track_id']

In [4]:
data.describe()

,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence
count,232725.000000,232725.000000,232725.000000,2.327250e+05,232725.000000,232725.000000,232725.000000,232725.000000,232725.000000,232725.000000,232725.000000
mean,41.127502,0.368560,0.554364,2.351223e+05,0.570958,0.148301,0.215009,-9.569885,0.120765,117.666585,0.454917
std,18.189948,0.354768,0.185608,1.189359e+05,0.263456,0.302768,0.198273,5.998204,0.185518,30.898907,0.260065
min,0.000000,0.000000,0.056900,1.538700e+04,0.000020,0.000000,0.009670,-52.457000,0.022200,30.379000,0.000000
25%,29.000000,0.037600,0.435000,1.828570e+05,0.385000,0.000000,0.097400,-11.771000,0.036700,92.959000,0.237000
50%,43.000000,0.232000,0.571000,2.204270e+05,0.605000,0.000044,0.128000,-7.762000,0.050100,115.778000,0.444000
75%,55.000000,0.722000,0.692000,2.657680e+05,0.787000,0.035800,0.264000,-5.501000,0.105000,139.054000,0.660000
max,100.000000,0.996000,0.989000,5.552917e+06,0.999000,0.999000,1.000000,3.744000,0.967000,242.903000,1.000000


First and foremost let's check for any 0 data points that we might need to replace:

In [6]:
""""
print(data['artist_name'].nunique())
artists = data.groupby('artist_name').count().iloc[:,0]
artists = artists.where(artists > 1).dropna()
artists
"""

'"\nprint(data[\'artist_name\'].nunique())\nartists = data.groupby(\'artist_name\').count().iloc[:,0]\nartists = artists.where(artists > 1).dropna()\nartists\n'

In [7]:
print(pd.isnull(data).sum())

genre               0
artist_name         0
track_name          0
popularity          0
acousticness        0
danceability        0
duration_ms         0
energy              0
instrumentalness    0
key                 0
liveness            0
loudness            0
mode                0
speechiness         0
tempo               0
time_signature      0
valence             0
dtype: int64


Fortunately there are none, let's move on with some of the feature engineering that we have done. 

Most values are numerical and need no preprocessing. We do have to convert however some text based rows into numbers which can be fed to the network afterwards.
Such rows are Key, Mode and Time Signature all of which will be replaced with integers 1 through the number of unique types that the given row may contain. 

I was not familiar with most of these terms, below you will find short descriptions that helped me better understand them.
Time signature: (also known as meter signature, metre signature, or measure signature) is a notational convention used in Western musical notation to specify how many beats (pulses) are contained in each measure (bar), and which note value is equivalent to a beat.
Mode: In the theory of Western music, it is a type of musical scale coupled with a set of characteristic melodic behaviors.

In [8]:
categorical_features = ["genre","artist_name","time_signature","key","mode"]
n_items = len(data)
for feat in categorical_features:
    print("Proccessing %s. number of unique fields: %d" % (feat, data[feat].nunique()))
    if data[feat].nunique()<50:
        print(data[feat].unique())
        print("Number of occurance of each unique value:")
        print(data.groupby(feat).count().iloc[:,0])
        for feat_value in data[feat].unique():
            if (len(data[data[feat]==feat_value]) / n_items <= 0.02):
                print("Adding %s category to the 'OTHER' category." % feat_value)
                data[feat] = data[feat].apply(lambda x: "OTHER" if x==feat_value else x, 1)
                
        print("Final number of unique fields:")
        print(data.groupby(feat).count().iloc[:,0]) 
        print("\n")

Proccessing genre. number of unique fields: 27
['Movie' 'R&B' 'A Capella' 'Alternative' 'Country' 'Dance' 'Electronic'
 'Anime' 'Folk' 'Blues' 'Opera' 'Hip-Hop' "Children's Music"
 'Children’s Music' 'Rap' 'Indie' 'Classical' 'Pop' 'Reggae' 'Reggaeton'
 'Jazz' 'Rock' 'Ska' 'Comedy' 'Soul' 'Soundtrack' 'World']
Number of occurance of each unique value:
genre
A Capella            119
Alternative         9263
Anime               8936
Blues               9023
Children's Music    5403
Children’s Music    9353
Classical           9256
Comedy              9681
Country             8664
Dance               8701
Electronic          9377
Folk                9299
Hip-Hop             9295
Indie               9543
Jazz                9441
Movie               7806
Opera               8280
Pop                 9386
R&B                 8992
Rap                 9232
Reggae              8771
Reggaeton           8927
Rock                9272
Ska                 8874
Soul                9089
Soundtrack     

As there are not too many possible values for these categorical features we can one-hot-encode them for more efficient learning, using pandas built-in function.

In [9]:
data = pd.get_dummies(data, columns=["genre", "time_signature", "key","mode"])
pd.set_option("max_columns",None)
data.sample(10)

,artist_name,track_name,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,genre_Alternative,genre_Anime,genre_Blues,genre_Children's Music,genre_Children’s Music,genre_Classical,genre_Comedy,genre_Country,genre_Dance,genre_Electronic,genre_Folk,genre_Hip-Hop,genre_Indie,genre_Jazz,genre_Movie,genre_OTHER,genre_Opera,genre_Pop,genre_R&B,genre_Rap,genre_Reggae,genre_Reggaeton,genre_Rock,genre_Ska,genre_Soul,genre_Soundtrack,genre_World,time_signature_3/4,time_signature_4/4,time_signature_5/4,time_signature_OTHER,key_A,key_A#,key_B,key_C,key_C#,key_D,key_D#,key_E,key_F,key_F#,key_G,key_G#,mode_Major,mode_Minor
103869,Flying Lotus,Do The Astral Plane,51,0.050500,0.722,237920,0.828,0.000000,0.1140,-3.227,0.1900,117.693,0.2750,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1
24247,Sumthin Sumthin,Stunt,39,0.006630,0.703,235333,0.872,0.175000,0.1180,-6.435,0.1500,90.048,0.3870,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
181919,The Disneyland Children's Chorus,"It's a Small World - From ""It's a Small World""",43,0.797000,0.678,182893,0.312,0.000000,0.1150,-14.695,0.0392,119.889,0.9530,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0
152944,John Legend,A Good Night,69,0.104000,0.530,203453,0.707,0.000000,0.1050,-5.516,0.2030,109.827,0.6960,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0
153187,Dropkick Murphys,The State Of Massachusetts,54,0.000945,0.398,232187,0.967,0.000006,0.1800,-3.232,0.0611,111.676,0.6110,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1
36711,Ganja White Night,Solera (feat. Clozee),28,0.002920,0.601,260569,0.654,0.754000,0.1100,-8.077,0.0332,127.997,0.0811,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1
124296,Giuseppe Verdi,"Falstaff, Act II Pt. 2: Fra poco s'incomincia ...",0,0.939000,0.499,102107,0.146,0.000003,0.4950,-24.238,0.1300,60.424,0.4030,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0
4637,Mark Wills,Where Did I Lose You,39,0.346000,0.476,225952,0.447,0.000028,0.0875,-6.434,0.0292,133.088,0.3240,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0
104698,Ludwig van Beethoven,"Diabelli Variations, Op. 120: Var. 9, Allegro ...",22,0.988000,0.389,105507,0.196,0.910000,0.2090,-23.298,0.0492,76.932,0.4420,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0
23503,1788-L,Pulsar Beam,42,0.000586,0.378,191294,0.933,0.013200,0.2190,-2.102,0.3140,170.653,0.3460,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0


Finally we split the data into 60% training 20% validation and 20% test subsets.

In [10]:
del data['artist_name']
del data['track_name']

In [11]:
train, validate, test = np.split(data.sample(frac=1), [int(.6*len(data)), int(.8*len(data))])

In [12]:
print(len(test), len(validate), len(train))

46545 46545 139635


In [13]:
## lets just play around with it, I don't want to lock my GPU for hours
demo_train = train[:10000]
demo_validate = validate[:2000]
demo_test = test[:2000]

In [14]:
#remove 'demo_' to run the training on the whole dataset.
Y_train = demo_train['popularity'].values #/ 100
Y_validate = demo_validate['popularity'].values# / 100
Y_test = demo_test['popularity'].values #/ 100

X_train = demo_train.drop(columns=['popularity'])
X_validate = demo_validate.drop(columns=['popularity'])
X_test = demo_test.drop(columns=['popularity'])

In [15]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_validate = scaler.transform(X_validate)
X_test = scaler.transform(X_test)

In [16]:
from keras.models import Sequential
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.layers import Dense, Dropout, Activation
from keras.models import load_model
from keras.optimizers import SGD

Using TensorFlow backend.
/home/daniel/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/daniel/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/daniel/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/daniel/.local/lib/python3.6/site-packages/tensorflow/python

In [18]:
#define callbacks

patience=20

early_stopping=EarlyStopping(patience=patience, verbose=1)
checkpointer=ModelCheckpoint(filepath='weights.hdf5', save_best_only=True, verbose=1)

model = Sequential()
model.add(Dense(output_dim=40, input_dim=X_train.shape[1]))
model.add(Activation('relu'))
model.add(Dense(output_dim=30))
model.add(Activation('relu'))
model.add(Dense(output_dim=1, activation='relu'))

#momentum can help us to get out of local minimums
sgd = SGD(lr=0.00001)
# using mse for regression problem
model.compile(loss='mse', optimizer=sgd)
history=model.fit(X_train,Y_train,epochs=100, 
                  batch_size=16,
                  verbose=2,
                  validation_data=(X_validate, Y_validate),
                  callbacks=[checkpointer, early_stopping])

/home/daniel/.local/lib/python3.6/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=55, units=40)`
  if __name__ == '__main__':
/home/daniel/.local/lib/python3.6/site-packages/ipykernel_launcher.py:11: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=30)`
  # This is added back by InteractiveShellApp.init_path()
/home/daniel/.local/lib/python3.6/site-packages/ipykernel_launcher.py:13: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=1)`
  del sys.path[0]


Train on 10000 samples, validate on 2000 samples
Epoch 1/100
 - 1s - loss: 1914.0832 - val_loss: 1642.0364

Epoch 00001: val_loss improved from inf to 1642.03639, saving model to weights.hdf5
Epoch 2/100
 - 1s - loss: 865.6629 - val_loss: 245.8320

Epoch 00002: val_loss improved from 1642.03639 to 245.83198, saving model to weights.hdf5
Epoch 3/100
 - 0s - loss: 176.4214 - val_loss: 137.3039

Epoch 00003: val_loss improved from 245.83198 to 137.30388, saving model to weights.hdf5
Epoch 4/100
 - 1s - loss: 127.4722 - val_loss: 115.0941

Epoch 00004: val_loss improved from 137.30388 to 115.09411, saving model to weights.hdf5
Epoch 5/100
 - 1s - loss: 113.7126 - val_loss: 107.3297

Epoch 00005: val_loss improved from 115.09411 to 107.32967, saving model to weights.hdf5
Epoch 6/100
 - 0s - loss: 108.0233 - val_loss: 103.5303

Epoch 00006: val_loss improved from 107.32967 to 103.53030, saving model to weights.hdf5
Epoch 7/100
 - 1s - loss: 104.9760 - val_loss: 101.5670

Epoch 00007: val_los

 - 1s - loss: 87.9039 - val_loss: 91.2750

Epoch 00061: val_loss improved from 91.33333 to 91.27503, saving model to weights.hdf5
Epoch 62/100
 - 1s - loss: 87.8154 - val_loss: 91.3384

Epoch 00062: val_loss did not improve from 91.27503
Epoch 63/100
 - 1s - loss: 87.7602 - val_loss: 91.3029

Epoch 00063: val_loss did not improve from 91.27503
Epoch 64/100
 - 1s - loss: 87.6788 - val_loss: 91.2877

Epoch 00064: val_loss did not improve from 91.27503
Epoch 65/100
 - 1s - loss: 87.6333 - val_loss: 91.2326

Epoch 00065: val_loss improved from 91.27503 to 91.23257, saving model to weights.hdf5
Epoch 66/100
 - 1s - loss: 87.5457 - val_loss: 91.2735

Epoch 00066: val_loss did not improve from 91.23257
Epoch 67/100
 - 1s - loss: 87.4851 - val_loss: 91.1121

Epoch 00067: val_loss improved from 91.23257 to 91.11214, saving model to weights.hdf5
Epoch 68/100
 - 0s - loss: 87.4364 - val_loss: 91.2275

Epoch 00068: val_loss did not improve from 91.11214
Epoch 69/100
 - 1s - loss: 87.3589 - val_los

In [19]:
# load the weights that performed best on the validation dataset
from sklearn.metrics import mean_squared_error
model = load_model('weights.hdf5')
# predict for the test dataset
preds = model.predict(X_test)
test_err = mean_squared_error(Y_test,preds)
print("Test error: ",test_err)

Test error:  98.97510559941371
